In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

zsh:1: command not found: apt-get


In [2]:
!wget https://archive.apache.org/dist/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
!tar xf spark-3.5.3-bin-hadoop3.tgz
!rm spark-3.5.3-bin-hadoop3.tgz   # Tidying up

--2025-03-04 10:24:54--  https://archive.apache.org/dist/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189
connected. to archive.apache.org (archive.apache.org)|65.108.204.189|:443... 
HTTP request sent, awaiting response... 200 OK
Length: 400864419 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.3-bin-hadoop3.tgz’

spark-3.5.3-bin-had 100%[===================>] 382.29M   255KB/s    in 33m 32s 

2025-03-04 10:58:26 (195 KB/s) - ‘spark-3.5.3-bin-hadoop3.tgz’ saved [400864419/400864419]



In [1]:
import os
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/jdk-21.jdk/Contents/Home"
os.environ["SPARK_HOME"] = "spark-3.5.3-bin-hadoop3"

In [2]:
!cd ~/CS/CovidComparison
!pwd

/Users/duncanbeauchamp/CS/CovidComparison


In [3]:
# !pip install -q findspark
import findspark
findspark.init()

In [4]:
import multiprocessing
print(multiprocessing.cpu_count())

8


In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("CSV Loader") \
    .master("local[*]") \
    .getOrCreate()

sc = spark.sparkContext  # Get the existing SparkContext

25/03/20 12:20:41 WARN Utils: Your hostname, Duncans-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.18.95.63 instead (on interface en0)
25/03/20 12:20:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/20 12:20:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/20 12:20:57 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [6]:
reddit_df = spark.read.csv("small_reddit.csv", header=True, inferSchema=True, multiLine=True, escape='"')

reddit_df.show()


+-------+--------------------+-----------+--------------------+-----+
|     id|      subreddit.name|created_utc|                body|score|
+-------+--------------------+-----------+--------------------+-----+
|hi1vsag|                nova| 1635206399|When you schedule...|    2|
|hi1vs7i|           vancouver| 1635206397|Didn't stop price...|   32|
|hi1vs5n|            pregnant| 1635206397|I’m just waiting ...|    1|
|hi1vs5v|            startrek| 1635206397|*The first duty o...|    1|
|hi1vs0l|             ontario| 1635206395|Compare BC to Ont...|   -2|
|hi1vrtk|       entertainment| 1635206393|Sheeran didn't me...|    2|
|hi1vrsk|          conspiracy| 1635206392|I see a lot of po...|    1|
|hi1vrnr|  lockdownskepticism| 1635206391|And think about h...|   29|
|hi1vrpy|                nova| 1635206391|Unfortunately the...|    2|
|hi1vrou|         antivaxxers| 1635206391|I don’t know anyo...|    1|
|hi1vrg2|          conspiracy| 1635206388|BS. Not a fear ca...|    1|
|hi1vre5|      toron

In [7]:
from pyspark.sql.functions import regexp_replace

# Assume df is your DataFrame with a "comment" column containing newlines
reddit_df_cleaned = reddit_df.withColumn("body", regexp_replace("body", "[\r\n]", " ")).select("id", "body")

reddit_df_cleaned.take(2)


[Row(id='hi1vsag', body='When you scheduled your booster with CVS does it just give you the option of Vaccines: COVID-19 (Vaccine brand) or does it specifically say booster'),
 Row(id='hi1vs7i', body="Didn't stop prices there though. New Zealand and Canada grew at about the same rate through COVID.   I agree that non-resident ownership should be stopped, but it's also probably not going to change anything.")]

In [9]:
reddit_df_cleaned.coalesce(1).write.csv("./reddit_test", header=True)

In [ ]:
twitter_df = spark.read.csv("small_twitter.csv", header=True, inferSchema=True, multiLine=True, escape='"')

twitter_df.show()

In [12]:
twitter_df_cleaned = twitter_df.withColumn("text", regexp_replace("text", "[\r\n]", " ")).select("status_id", "lang", "text")
# twitter_df_cleaned2 = twitter_df_cleaned.select("status_id", "lang", "text")

twitter_df_cleaned.take(2)

[Row(status_id=1244051646071611394, lang='es', text='Ante cualquier enfermedad respiratoria, no te automediques. #PrevenciónCoronavirus #Coronavirus #COVID19 https://t.co/IgB2izYi1p'),
 Row(status_id=1244051645039706112, lang='es', text='#ATENCIÓN En el Terminal Nuevo Circo se implementan medidas como el uso de la mascarilla por parte de usuari@s y conductores que hacen vida en dichas instalaciones, parte de los esfuerzos unificados para impedir la propagación del #Coronavirus #28Marzo @NicolasMaduro  @ErikaPSUV https://t.co/yD8g0EAcQz')]

In [13]:
twitter_df_cleaned.coalesce(1).write.csv("./new_twitter_test", header=True)

In [29]:
reddit_rdd = sc.textFile("test/cleaned_small_reddit.csv")

In [ ]:
%%time

pos_words_set = set()
with open('sentiment_txts/positive_words_en.txt','r') as file:
    for line in file:
        pos_words_set.add(line.strip())
print(pos_words_set)


In [32]:
rdd.take(5)

['id,subreddit.name,created_utc,body,score',
 'hi1vsag,nova,1635206399,When you scheduled your booster with CVS does it just give you the option of Vaccines: COVID-19 (Vaccine brand) or does it specifically say booster,2',
 'hi1vs7i,vancouver,1635206397,"Didn\'t stop prices there though. New Zealand and Canada grew at about the same rate through COVID.   I agree that non-resident ownership should be stopped, but it\'s also probably not going to change anything.",32',
 'hi1vs5n,pregnant,1635206397,I’m just waiting until after pregnancy to get vaccinated.  I have a special needs daughter due to having fever during pregnancy and I’m not willing to risk the vaccine fever right now. I have strong antibodies from Covid that I had before I got pregnant. (I keep taking blood tests to check) There’s a difference between anti vaccine and deciding to wait based on different factors,1',
 'hi1vs5v,startrek,1635206397,"*The first duty of every Starfleet officer is to the truth. Whether it\'s scienti